In [4]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

In [5]:
link_list = []
for n in range(1, 15):
    link = 'https://corporate.tesco.hu/media/sajt%C3%B3koezlem%C3%A9nyek/?year=0&categoryId=0&author=0&page=' + str(n)
    r1 = requests.get(link)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html5lib')
    coverpage_news = soup1.find_all('li', class_="listing__item")
    for l in range(len(coverpage_news)):
        link_list.append("https://corporate.tesco.hu/" +  coverpage_news[l].find('a')['href'])

len(link_list)

140

### Van egy listám, ami minden idevágó cikk linkjét tartalmazza. (link_list)

In [6]:
cikkek_adatai = []
start = time.time()
counter = 0
for m in link_list:
    # link
    article = requests.get(m)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')

    # dátum
    date_raw = soup_article.select(".article-date")
    date = " "
    if len(date_raw)>0: 
        date = date_raw[0].get_text()

    # cím
    title_raw = soup_article.select("h1")
    title = title_raw[0].get_text()

    # törzsszöveg
    body = soup_article.select(".rte-wrapper")
    x = body[1].select('p')
    list_paragraphs = []
    for p in np.arange(1, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
    
    # putting it together
    cikkek_adatai.append([date, title, final_article, m])
    end = time.time()
    counter += 1
    print(str(counter)+"/140", end - start)



1/140 20.704549312591553
2/140 38.64437913894653
3/140 56.7412474155426
4/140 75.51424860954285
5/140 93.23068046569824
6/140 110.347736120224
7/140 130.60825896263123
8/140 149.92668104171753
9/140 167.3051242828369
10/140 184.92279982566833
11/140 202.06081557273865
12/140 219.62595582008362
13/140 237.78581380844116
14/140 257.00911355018616
15/140 273.83818554878235
16/140 292.1467909812927
17/140 310.6536281108856
18/140 327.4054477214813
19/140 344.4533987045288
20/140 363.47649788856506
21/140 381.20685172080994
22/140 397.93464756011963
23/140 414.29682874679565
24/140 432.3778386116028
25/140 450.08551716804504
26/140 468.1028652191162
27/140 485.45585536956787
28/140 502.4859449863434
29/140 519.4836266040802
30/140 535.9454681873322
31/140 554.6700973510742
32/140 573.1096088886261
33/140 590.9619493484497
34/140 608.4457287788391
35/140 625.4109394550323
36/140 643.7495837211609
37/140 661.6345973014832
38/140 679.5416352748871
39/140 697.2377955913544
40/140 716.9736094474

In [10]:
df = pd.DataFrame(data=cikkek_adatai)
df.columns = ['dátaum', 'cím', 'szöveg', 'link']
with pd.ExcelWriter('tesco.xlsx') as writer:  
    df.to_excel(writer, sheet_name='hu')